In [2]:
!pip install inference

In [4]:
!pip install ultralytics

In [5]:
!pip install supervision==0.19.0

In [6]:
import gdown
import tqdm
import ultralytics
import inference
import supervision  #==0.19.0

In [7]:
pwd

'/content'

In [9]:
!mkdir proj_supervision

In [10]:
!mkdir proj_supervision/data

In [11]:
# DATA DOWNLOAD
url = "https://drive.google.com/uc?id=1vVrEVMxucHgqGd7vAa501ASojbeGPhIr"
#DIR = "/content/proj_supervision"
output = "/content/proj_supervision/data/market-square.mp4"
gdown.download(url, output)

Downloading...
From: https://drive.google.com/uc?id=1vVrEVMxucHgqGd7vAa501ASojbeGPhIr
To: /content/proj_supervision/data/market-square.mp4
100%|██████████| 21.3M/21.3M [00:00<00:00, 225MB/s]


'/content/proj_supervision/data/market-square.mp4'

In [13]:
cd proj_supervision/data/

/content/proj_supervision/data


In [14]:
#upload 4 files from "C:\Users\Julia\Desktop\cv\git\resources\supervision\examples\count_people_in_zone\data" to folder "data"
from google.colab import files
uploaded = files.upload()

Saving horizontal-zone-config.json to horizontal-zone-config.json
Saving multi-zone-config.json to multi-zone-config.json
Saving quarters-zone-config.json to quarters-zone-config.json
Saving vertical-zone-config.json to vertical-zone-config.json


In [15]:
cd /content

/content


In [16]:
%%writefile parsing.py
##### SCRIPT STARTS HERE #####
#!usr/bin/bash python

import argparse

# Define the ArgumentParser
parser = argparse.ArgumentParser()

# Indicate end of argument definitions and parse args
parser.parse_args()

Writing parsing.py


In [17]:
# !python inference_example.py

In [19]:
import argparse
import json
import os
from typing import List, Tuple

import cv2
import numpy as np
from inference.core.models.roboflow import RoboflowInferenceModel
from inference.models.utils import get_roboflow_model
from tqdm import tqdm

import supervision as sv

COLORS = sv.ColorPalette.default()

SupervisionWarnings: default is deprecated: `ColorPalette.default()` is deprecated and will be removed in `supervision-0.22.0`. Use `Color.DEFAULT` instead.


In [20]:
def load_zones_config(file_path: str) -> List[np.ndarray]:
    """
    Load polygon zone configurations from a JSON file.

    This function reads a JSON file which contains polygon coordinates, and
    converts them into a list of NumPy arrays. Each polygon is represented as
    a NumPy array of coordinates.

    Args:
    file_path (str): The path to the JSON configuration file.

    Returns:
    List[np.ndarray]: A list of polygons, each represented as a NumPy array.
    """
    with open(file_path, "r") as file:
        data = json.load(file)
        return [np.array(polygon, np.int32) for polygon in data["polygons"]]

def initiate_annotators(
    polygons: List[np.ndarray], resolution_wh: Tuple[int, int]
) -> Tuple[
    List[sv.PolygonZone], List[sv.PolygonZoneAnnotator], List[sv.BoundingBoxAnnotator]
]:
    line_thickness = sv.calculate_dynamic_line_thickness(resolution_wh=resolution_wh)
    text_scale = sv.calculate_dynamic_text_scale(resolution_wh=resolution_wh)

    zones = []
    zone_annotators = []
    box_annotators = []

    for index, polygon in enumerate(polygons):
        zone = sv.PolygonZone(polygon=polygon, frame_resolution_wh=resolution_wh)
        zone_annotator = sv.PolygonZoneAnnotator(
            zone=zone,
            color=COLORS.by_idx(index),
            thickness=line_thickness,
            text_thickness=line_thickness * 2,
            text_scale=text_scale * 2,
        )
        box_annotator = sv.BoundingBoxAnnotator(
            color=COLORS.by_idx(index), thickness=line_thickness
        )
        zones.append(zone)
        zone_annotators.append(zone_annotator)
        box_annotators.append(box_annotator)

    return zones, zone_annotators, box_annotators

def detect(
    frame: np.ndarray, model: RoboflowInferenceModel, confidence_threshold: float = 0.5
) -> sv.Detections:
    """
    Detect objects in a frame using Inference model, filtering detections by class ID
        and confidence threshold.

    Args:
        frame (np.ndarray): The frame to process, expected to be a NumPy array.
        model (RoboflowInferenceModel): The Inference model used for processing the
            frame.
        confidence_threshold (float, optional): The confidence threshold for filtering
            detections. Default is 0.5.

    Returns:
        sv.Detections: Filtered detections after processing the frame with the Inference
            model.

    Note:
        This function is specifically tailored for an Inference model and assumes class
        ID 0 for filtering.
    """
    results = model.infer(frame)[0]
    detections = sv.Detections.from_inference(results)
    filter_by_class = detections.class_id == 0
    filter_by_confidence = detections.confidence > confidence_threshold
    return detections[filter_by_class & filter_by_confidence]

def annotate(
    frame: np.ndarray,
    zones: List[sv.PolygonZone],
    zone_annotators: List[sv.PolygonZoneAnnotator],
    box_annotators: List[sv.BoundingBoxAnnotator],
    detections: sv.Detections,
) -> np.ndarray:
    """
    Annotate a frame with zone and box annotations based on given detections.

    Args:
        frame (np.ndarray): The original frame to be annotated.
        zones (List[sv.PolygonZone]): A list of polygon zones used for detection.
        zone_annotators (List[sv.PolygonZoneAnnotator]): A list of annotators for
            drawing zone annotations.
        box_annotators (List[sv.BoundingBoxAnnotator]): A list of annotators for
            drawing box annotations.
        detections (sv.Detections): Detections to be used for annotation.

    Returns:
        np.ndarray: The annotated frame.
    """
    annotated_frame = frame.copy()
    for zone, zone_annotator, box_annotator in zip(
        zones, zone_annotators, box_annotators
    ):
        detections_in_zone = detections[zone.trigger(detections=detections)]
        annotated_frame = zone_annotator.annotate(scene=annotated_frame)
        annotated_frame = box_annotator.annotate(
            scene=annotated_frame, detections=detections_in_zone
        )
    return annotated_frame


In [28]:
%%writefile parsing.py
##### SCRIPT STARTS HERE #####
#!usr/bin/bash python

import argparse

# Define the ArgumentParser
parser = argparse.ArgumentParser()

# Indicate end of argument definitions and parse args
parser.parse_args()

Writing parsing.py


In [48]:
source_video_path = "/content/proj_supervision/data/market-square.mp4"
roboflow_api_key = None
zone_configuration_path = "/content/proj_supervision/data/horizontal-zone-config.json"
model_id = "yolov8x-1280"
target_video_path = "/content/proj_supervision/data/market-square-res.mp4"
confidence_threshold = 0.3
iou_threshold = 0.7

In [46]:
api_key = roboflow_api_key
api_key = os.environ.get("ROBOFLOW_API_KEY", api_key)
if api_key is None:
        raise ValueError(
            "Roboflow API key is missing. Please provide it as an argument or set the "
            "ROBOFLOW_API_KEY environment variable."
        )
roboflow_api_key = api_key

ValueError: Roboflow API key is missing. Please provide it as an argument or set the ROBOFLOW_API_KEY environment variable.

In [49]:
video_info = sv.VideoInfo.from_video_path(source_video_path)
polygons = load_zones_config(zone_configuration_path)
zones, zone_annotators, box_annotators = initiate_annotators(
        polygons=polygons, resolution_wh=video_info.resolution_wh
    )


In [50]:
model = get_roboflow_model(model_id=model_id, api_key=roboflow_api_key)

In [51]:
frames_generator = sv.get_video_frames_generator(source_video_path)

In [54]:
if target_video_path is not None:
    with sv.VideoSink(target_video_path, video_info) as sink:
          for frame in tqdm(frames_generator, total=video_info.total_frames):
              detections = detect(frame, model, confidence_threshold)
              annotated_frame = annotate(
                  frame=frame,
                  zones=zones,
                  zone_annotators=zone_annotators,
                  box_annotators=box_annotators,
                  detections=detections,
              )
              sink.write_frame(annotated_frame)
else:
    for frame in tqdm(frames_generator, total=video_info.total_frames):
          detections = detect(frame, model, confidence_threshold)
          annotated_frame = annotate(
            frame=frame,
            zones=zones,
            zone_annotators=zone_annotators,
            box_annotators=box_annotators,
            detections=detections,
          )
          cv2.imshow("Processed Video", annotated_frame)
          if cv2.waitKey(1) & 0xFF == ord("q"):
            break

    cv2.destroyAllWindows()

100%|██████████| 474/474 [2:42:04<00:00, 20.52s/it]
